<a href="https://colab.research.google.com/github/lima-breno/deep_learning_frameworks/blob/main/MEXIDOframeworks_keras(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **FRAMEWORKS DE DEEPLEARNING**
# **Prática - Keras**

**Descrição**: Este notebook apresenta um exemplo de uma rede neural profunda com mais de uma camada para um problema de classificação.


# **Saúde Fetal**

As Cardiotocografias (CTGs) são opções simples e de baixo custo para avaliar a saúde fetal, permitindo que os profissionais de saúde atuem na prevenção da mortalidade infantil e materna. O próprio equipamento funciona enviando pulsos de ultrassom e lendo sua resposta, lançando luz sobre a frequência cardíaca fetal (FCF), movimentos fetais, contrações uterinas e muito mais.

Este conjunto de dados contém 2126 registros de características extraídas de exames de Cardiotocografias, que foram então classificados por três obstetras especialistas em 3 classes:

- Normal
- Suspeito
- Patológico

# 1 - Importando os módulos necessários

In [ ]:
import os
import tensorflow
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, InputLayer, Dropout
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.regularizers import l1, l2

import random
import numpy as np
import random as python_random
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Definindo funções adicionais

In [ ]:
def plot_convergence(hist, metric):

  df = pd.DataFrame(hist.history)
  fig = px.line(df,
                x=np.arange(df[metric].shape[0]),
                y=[f'{metric}', f'val_{metric}'])
  fig.show()

def reset_seeds():
   os.environ['PYTHONHASHSEED']=str(42)
   tf.random.set_seed(42)
   np.random.seed(42)
   random.seed(42)

# 2 - Fazendo a leitura do dataset e atribuindo às respectivas variáveis

In [ ]:
data = pd.read_csv('https://raw.githubusercontent.com/renansantosmendes/lectures-cdas-2023/master/fetal_health.csv')

# 3 - Preparando o dado antes de iniciar o treino do modelo

In [ ]:
data.head()

,baseline value,accelerations,fetal_movement,uterine_contractions,light_decelerations,severe_decelerations,prolongued_decelerations,abnormal_short_term_variability,mean_value_of_short_term_variability,percentage_of_time_with_abnormal_long_term_variability,...,histogram_min,histogram_max,histogram_number_of_peaks,histogram_number_of_zeroes,histogram_mode,histogram_mean,histogram_median,histogram_variance,histogram_tendency,fetal_health
0,120.0,0.000,0.0,0.000,0.000,0.0,0.0,73.0,0.5,43.0,...,62.0,126.0,2.0,0.0,120.0,137.0,121.0,73.0,1.0,2.0
1,132.0,0.006,0.0,0.006,0.003,0.0,0.0,17.0,2.1,0.0,...,68.0,198.0,6.0,1.0,141.0,136.0,140.0,12.0,0.0,1.0
2,133.0,0.003,0.0,0.008,0.003,0.0,0.0,16.0,2.1,0.0,...,68.0,198.0,5.0,1.0,141.0,135.0,138.0,13.0,0.0,1.0
3,134.0,0.003,0.0,0.008,0.003,0.0,0.0,16.0,2.4,0.0,...,53.0,170.0,11.0,0.0,137.0,134.0,137.0,13.0,1.0,1.0
4,132.0,0.007,0.0,0.008,0.000,0.0,0.0,16.0,2.4,0.0,...,53.0,170.0,9.0,0.0,137.0,136.0,138.0,11.0,1.0,1.0


In [ ]:
data["fetal_health"].head(6)

,fetal_health
0,2.0
1,1.0
2,1.0
3,1.0
4,1.0
5,3.0


```
        N(1)  S(2)  D(3)
array([[0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.]])
```

In [ ]:
# [1.2, 13.6, 2.5] -> [0, 1, 0] -> [BB, C, A]

In [ ]:
y_pred = np.array([0, 1, 0])

In [ ]:
y_pred

array([0, 1, 0])

In [ ]:
str(np.argmax(y_pred))

NameError: name 'y_pred' is not defined

In [ ]:
dict_response = {
    '0': 'BB',
    '1': 'C',
    '2': 'A'
}

y -> [0, 0, 1]

In [ ]:
dict_response.get(str(np.argmax(np.array([0.1, 0.2, 0.7]))))

'A'

In [ ]:
np.array([0.1, 0.2, 0.7]) - np.array([ 0.0, 0.0, 1.0])

array([ 0.1,  0.2, -0.3])

In [ ]:
to_categorical(data["fetal_health"].head(6))

array([[0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.]])

In [ ]:
data["fetal_health"].head(6) - 1

,fetal_health
0,1.0
1,0.0
2,0.0
3,0.0
4,0.0
5,2.0


In [ ]:
to_categorical(data["fetal_health"].head(6) - 1)

array([[0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.]])

In [ ]:
X=data.drop(["fetal_health"], axis=1)
y=data["fetal_health"] - 1

columns_names = list(X.columns)
X_df = preprocessing.StandardScaler().fit_transform(X)
X_df = pd.DataFrame(X_df, columns=columns_names)

X_train, X_test, y_train, y_test = train_test_split(X_df, y, test_size=0.3, random_state=42)

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# 4 - Criando o modelo e adicionando as camadas

In [ ]:
input_shape = X_train.iloc[0].shape

In [ ]:
reset_seeds()
model = Sequential()
model.add(Dense(units=30, activation='relu', input_shape=input_shape))
model.add(Dense(units=30, activation='relu'))
model.add(Dense(units=30, activation='relu'))
model.add(Dense(units=3, activation='softmax'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


OBS: rede neural  softmax faz a categorização, para poder tornar o dado mais entendível, transformando em probabilidade

# 5 - Compilando o modelo


In [ ]:
model.compile(loss='categorical_crossentropy',
              metrics=['accuracy'])

# 6 - Executando o treino do modelo

In [ ]:
%%time
hist = model.fit(x=X_train,
                 y=y_train,
                 validation_split = 0.2, #Aqui posso acrescentar pro keras validar um percentual
                 epochs=30)

Epoch 1/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.6708 - loss: 0.8789 - val_accuracy: 0.8221 - val_loss: 0.5318
Epoch 2/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8004 - loss: 0.5265 - val_accuracy: 0.8725 - val_loss: 0.3528
Epoch 3/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8362 - loss: 0.3956 - val_accuracy: 0.8993 - val_loss: 0.2864
Epoch 4/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8623 - loss: 0.3377 - val_accuracy: 0.9060 - val_loss: 0.2581
Epoch 5/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8777 - loss: 0.3048 - val_accuracy: 0.9027 - val_loss: 0.2423
Epoch 6/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8943 - loss: 0.2821 - val_accuracy: 0.9027 - val_loss: 0.2325
Epoch 7/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9032 - loss: 0.2648 - val_accuracy: 0.9027 - val_loss: 0.2248
Epoch 8/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9052 - loss: 0.2512 - val_accuracy: 0.9060 - val_loss

CUidado com o treinamento, pois se vc rodar mais de uma vez (sem resetar os dados) ele vai maquiar os pesos/resultados.

# 6.1 Avaliando o modelo

# 6.2 Plotando a acurácia do modelo

In [ ]:
plot_convergence(
    hist=hist,
    metric ='accuracy'
    )

o grafico varia em épocas(horizontal) e a acurácia(Vertical)


In [ ]:
#a função perda dá o valor real de qd o seu modelo está errando
plot_convergence(
    hist=hist,
    metric ='loss'
    )

In [ ]:
#o grafico acima diz que função de perda mede o quanto distante o modelo está da saída esperada.

In [ ]:
model.evaluate(
    x=X_test,
    y=y_test,
    batch_size = X_test.shape[0] # aqui ele considera os dados por completo.
    ) #usa a metrica e a funcao de perda do dataset que passei para ele
  # aqui ele avalia considerando o batch completo!

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - accuracy: 0.9028 - loss: 0.2303


[0.2302897572517395, 0.9028213024139404]

In [ ]:
#SE quero fazer uma previsão, de acordo com dados recebidos do momento (considernado como se fosse um dado aleatorio - mas já tratatdo, onde vc precisa somente
##fazer a previsão)

pregnant = X_train.iloc[100]
##só faço uma alteração do dado (de 21 linhas e 1 coluna para 1 linha e 21 colunas)

exam_data = pregnant.values.reshape(1,-1)

In [ ]:
y_train[100]
#Aqui o bebe é normal

array([1., 0., 0.])

In [ ]:
#usando a função do modelo para prever  (fazendo a leitura do exame)
prediction = model.predict(exam_data)
rounded_predictions = np.round(prediction,4)
print(rounded_predictions)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
[[9.862e-01 1.320e-02 6.000e-04]]


In [ ]:
#como interpretar essa saída??
''' a minha array lá em cima tem 3 dados, o bebe normal, com possivel problema  e com sofrimento fetal,
a minha rede neural tem a ultima camada com 3 neuronios (3 classes). Logo, os dados apresentados como resposta
sao similares a um array [1, 0, 0] -> O bb é saudável (conforme o array do iloc[100])'''


' a minha array lá em cima tem 3 dados, o bebe normal, com possivel problema  e com sofrimento fetal,\na minha rede neural tem a ultima camada com 3 neuronios (3 classes). Logo, os dados apresentados como resposta\nsao similares a um array [1, 0, 0] -> O bb é saudável (conforme o array do iloc[100])'

In [ ]:
int(np.argmax(prediction))
#função que pega o maior valor do predictions e identifica a posição dele. Isso autotmaticamente já o classifica



0

In [ ]:
#é classe zero! o primeiro elemento[0] da array, classificando como Saudável!

In [ ]:
#cuidado, pois a probabilidade pode ser quebrada ex: [0.49, 0.51, 0] --> isso é um problema na hr de terceirizar a decisao para o modelo, pois pode diagnosticar
## de maneira errônea, o modelo encaixaria o paciente como possível doente. Se for meio a meio, ele pega o primeiro termo

# o ideal é colocar um condicional para evitar estes problema (ex: treshold)

prediction_1 = np.array([0.49, 0.51])
predicted_class = int(np.argmax(prediction_1))
threshold = 0.95 # para o paciente ter que fazer o tratamento

if predicted_class == 1 and float(prediction_1[predicted_class]) > threshold:
  print('fazer tratamento')
else:
   print('saudável')

saudável


In [ ]:
#como mostrar o resumo do modelo dentro do keras

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 30)             │           660 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 30)             │           930 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 30)             │           930 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 3)              │            93 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,228 (20.43 KB)

 Trainable params: 2,613 (10.21 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,615 (10.22 KB)

In [ ]:
#ele esconde a camada de entrada. Ele vai empilhando as camadas densas, passa a qtd de neuronios de saida (30, 30, 30  e 3).
##cada neuronio tem o peso de entrada mais o bias, logo, os parametros da segunda camada sao (30*30)+30 = 930


##7. Correções de Overfitting

In [ ]:
input_shape = X_train.iloc[0].shape
reset_seeds()
model = Sequential()
model.add(Dense(units=100, activation='relu', input_shape=input_shape))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=300, activation='relu'))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=300, activation='relu'))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=3, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              metrics=['accuracy'])

print(model.summary())
hist = model.fit(x=X_train,
                 y=y_train,
                 validation_split = 0.2, #Aqui posso acrescentar pro keras validar um percentual
                 epochs=30,
                 verbose = 3)

plot_convergence(
    hist=hist,
    metric ='loss'
    )

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_12 (Dense)                │ (None, 100)            │         2,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 100)            │        10,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 100)            │        10,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 300)            │        30,300 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 100)            │        30,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 300)            │        30,300 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 100)            │        30,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 3)              │           303 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 143,503 (560.56 KB)

 Trainable params: 143,503 (560.56 KB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/30
Epoch 2/30
Epoch 3/30
Epoch 4/30
Epoch 5/30
Epoch 6/30
Epoch 7/30
Epoch 8/30
Epoch 9/30
Epoch 10/30
Epoch 11/30
Epoch 12/30
Epoch 13/30
Epoch 14/30
Epoch 15/30
Epoch 16/30
Epoch 17/30
Epoch 18/30
Epoch 19/30
Epoch 20/30
Epoch 21/30
Epoch 22/30
Epoch 23/30
Epoch 24/30
Epoch 25/30
Epoch 26/30
Epoch 27/30
Epoch 28/30
Epoch 29/30
Epoch 30/30


In [ ]:
#aqui o modelo deu ruim demais! Modelo ficou com muitos parametros 143,503
'''a curva de treino do modelo oscilou mas até que a função perda está diminuindo. Se ficarmos somente nisso, aparentemente o modelo deu tudo certo!
Só que não!!

Mas, ao verificarmos a curva de validação da função perda verificamos que está aumentando! Ele está ficando pior nos dados que ele não viu! Foi dada muita capacidade pra ele, aí perde a mão.
analogia: é como se o modelo decorasse os dados de treino (se especializa demais com os dados), ai como foge muito dos dados de treino ele se perde. chamado de overfitting!
Oou seja, é um modelo parrudo demais para um dado muito simples

SACADA PARA ESSE TIPO DE SITUAÇÃO
hipotese 1) simplificar o modelo (diminuir qtd de neuronios)

'''
#diminuindo qtd de neuronios
input_shape = X_train.iloc[0].shape
reset_seeds()
model = Sequential()
model.add(Dense(units=100, activation='relu', input_shape=input_shape))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=3, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              metrics=['accuracy'])

print(model.summary())
hist = model.fit(x=X_train,
                 y=y_train,
                 validation_split = 0.2, #Aqui posso acrescentar pro keras validar um percentual
                 epochs=30,
                 verbose = 3)

plot_convergence(
    hist=hist,
    metric ='loss'
    )
#aqui já baixa para 56k só que ainda deu ruim!

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_28 (Dense)                │ (None, 100)            │         2,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_29 (Dense)                │ (None, 100)            │        10,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_30 (Dense)                │ (None, 100)            │        10,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_31 (Dense)                │ (None, 100)            │        10,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_32 (Dense)                │ (None, 100)            │        10,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_33 (Dense)                │ (None, 100)            │        10,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_34 (Dense)                │ (None, 100)            │        10,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_35 (Dense)                │ (None, 3)              │           303 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 63,103 (246.50 KB)

 Trainable params: 63,103 (246.50 KB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/30
Epoch 2/30
Epoch 3/30
Epoch 4/30
Epoch 5/30
Epoch 6/30
Epoch 7/30
Epoch 8/30
Epoch 9/30
Epoch 10/30
Epoch 11/30
Epoch 12/30
Epoch 13/30
Epoch 14/30
Epoch 15/30
Epoch 16/30
Epoch 17/30
Epoch 18/30
Epoch 19/30
Epoch 20/30
Epoch 21/30
Epoch 22/30
Epoch 23/30
Epoch 24/30
Epoch 25/30
Epoch 26/30
Epoch 27/30
Epoch 28/30
Epoch 29/30
Epoch 30/30


In [ ]:
#diminuindo qtd de camadas neuronios (removendo 2 camadas) - baixa paa 42k paranetris
input_shape = X_train.iloc[0].shape
reset_seeds()
model = Sequential()
model.add(Dense(units=100, activation='relu', input_shape=input_shape))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=3, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              metrics=['accuracy'])

print(model.summary())
hist = model.fit(x=X_train,
                 y=y_train,
                 validation_split = 0.2, #Aqui posso acrescentar pro keras validar um percentual
                 epochs=30,
                 verbose = 3)

plot_convergence(
    hist=hist,
    metric ='loss'
    )

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_42 (Dense)                │ (None, 100)            │         2,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_43 (Dense)                │ (None, 100)            │        10,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_44 (Dense)                │ (None, 100)            │        10,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_45 (Dense)                │ (None, 100)            │        10,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_46 (Dense)                │ (None, 100)            │        10,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_47 (Dense)                │ (None, 3)              │           303 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 42,903 (167.59 KB)

 Trainable params: 42,903 (167.59 KB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/30
Epoch 2/30
Epoch 3/30
Epoch 4/30
Epoch 5/30
Epoch 6/30
Epoch 7/30
Epoch 8/30
Epoch 9/30
Epoch 10/30
Epoch 11/30
Epoch 12/30
Epoch 13/30
Epoch 14/30
Epoch 15/30
Epoch 16/30
Epoch 17/30
Epoch 18/30
Epoch 19/30
Epoch 20/30
Epoch 21/30
Epoch 22/30
Epoch 23/30
Epoch 24/30
Epoch 25/30
Epoch 26/30
Epoch 27/30
Epoch 28/30
Epoch 29/30
Epoch 30/30


In [ ]:
#diminuindo qtd de camadas (agora 4 camadas) - baixou para 22k parametros
input_shape = X_train.iloc[0].shape
reset_seeds()
model = Sequential()
model.add(Dense(units=100, activation='relu', input_shape=input_shape))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=3, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              metrics=['accuracy'])

print(model.summary())
hist = model.fit(x=X_train,
                 y=y_train,
                 validation_split = 0.2, #Aqui posso acrescentar pro keras validar um percentual
                 epochs=30,
                 verbose = 3)

plot_convergence(
    hist=hist,
    metric ='loss'
    )


'''
hipotese 2) reduzir a qtd de parametros ou dificultar a vida do modelo

(obs: existe uma relação entre capacidade de processamento do modelo e qtd de parametros)




'''

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_52 (Dense)                │ (None, 100)            │         2,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_53 (Dense)                │ (None, 100)            │        10,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_54 (Dense)                │ (None, 100)            │        10,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_55 (Dense)                │ (None, 3)              │           303 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 22,703 (88.68 KB)

 Trainable params: 22,703 (88.68 KB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/30
Epoch 2/30
Epoch 3/30
Epoch 4/30
Epoch 5/30
Epoch 6/30
Epoch 7/30
Epoch 8/30
Epoch 9/30
Epoch 10/30
Epoch 11/30
Epoch 12/30
Epoch 13/30
Epoch 14/30
Epoch 15/30
Epoch 16/30
Epoch 17/30
Epoch 18/30
Epoch 19/30
Epoch 20/30
Epoch 21/30
Epoch 22/30
Epoch 23/30
Epoch 24/30
Epoch 25/30
Epoch 26/30
Epoch 27/30
Epoch 28/30
Epoch 29/30
Epoch 30/30


'\nhipotese 2) reduzir a qtd de parametros ou dificultar a vida do modelo\n\n(obs: existe uma relação entre capacidade de processamento do modelo e qtd de parametros)\n\n\n\n\n'

In [ ]:
#diminuindo a qtd de camadas (removendo 4) e diminuindo a qtd de neuronios - ja baixa para 6k, fica melhoe
input_shape = X_train.iloc[0].shape
reset_seeds()
model = Sequential()
model.add(Dense(units=50, activation='relu', input_shape=input_shape))
model.add(Dense(units=50, activation='relu'))
model.add(Dense(units=50, activation='relu'))
model.add(Dense(units=3, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              metrics=['accuracy'])

print(model.summary())
hist = model.fit(x=X_train,
                 y=y_train,
                 validation_split = 0.2, #Aqui posso acrescentar pro keras validar um percentual
                 epochs=30,
                 verbose = 3)

plot_convergence(
    hist=hist,
    metric ='loss'
    )

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_60 (Dense)                │ (None, 50)             │         1,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_61 (Dense)                │ (None, 50)             │         2,550 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_62 (Dense)                │ (None, 50)             │         2,550 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_63 (Dense)                │ (None, 3)              │           153 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,353 (24.82 KB)

 Trainable params: 6,353 (24.82 KB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/30
Epoch 2/30
Epoch 3/30
Epoch 4/30
Epoch 5/30
Epoch 6/30
Epoch 7/30
Epoch 8/30
Epoch 9/30
Epoch 10/30
Epoch 11/30
Epoch 12/30
Epoch 13/30
Epoch 14/30
Epoch 15/30
Epoch 16/30
Epoch 17/30
Epoch 18/30
Epoch 19/30
Epoch 20/30
Epoch 21/30
Epoch 22/30
Epoch 23/30
Epoch 24/30
Epoch 25/30
Epoch 26/30
Epoch 27/30
Epoch 28/30
Epoch 29/30
Epoch 30/30


Hipotese 2: penalizando o modelo
L1: Lasso
L2: Reach

In [ ]:
#HIPOTESE 2: penalizando o modelo

%%time
input_shape = X_train.iloc[0].shape
reset_seeds()
model = Sequential()
model.add(Dense(units=100, activation='relu', input_shape=input_shape))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=300, activation='relu'))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=300, activation='relu'))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=3, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              metrics=['accuracy'])

print(model.summary())
hist = model.fit(x=X_train,
                 y=y_train,
                 validation_split = 0.2, #Aqui posso acrescentar pro keras validar um percentual
                 epochs=30,
                 verbose = 3)

plot_convergence(
    hist=hist,
    metric ='loss'
    )

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_64 (Dense)                │ (None, 100)            │         2,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_65 (Dense)                │ (None, 100)            │        10,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_66 (Dense)                │ (None, 100)            │        10,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_67 (Dense)                │ (None, 300)            │        30,300 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_68 (Dense)                │ (None, 100)            │        30,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_69 (Dense)                │ (None, 300)            │        30,300 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_70 (Dense)                │ (None, 100)            │        30,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_71 (Dense)                │ (None, 3)              │           303 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 143,503 (560.56 KB)

 Trainable params: 143,503 (560.56 KB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/30
Epoch 2/30
Epoch 3/30
Epoch 4/30
Epoch 5/30
Epoch 6/30
Epoch 7/30
Epoch 8/30
Epoch 9/30
Epoch 10/30
Epoch 11/30
Epoch 12/30
Epoch 13/30
Epoch 14/30
Epoch 15/30
Epoch 16/30
Epoch 17/30
Epoch 18/30
Epoch 19/30
Epoch 20/30
Epoch 21/30
Epoch 22/30
Epoch 23/30
Epoch 24/30
Epoch 25/30
Epoch 26/30
Epoch 27/30
Epoch 28/30
Epoch 29/30
Epoch 30/30


CPU times: user 11.9 s, sys: 609 ms, total: 12.5 s
Wall time: 14.3 s


In [ ]:
#posso aplicar a correção nas camadas que tem a maior qtd de parametros (esta info está na tabela do modelo dos parametros)

#Para issoo, importar do keras as funções que permitem fazer a correção
##L1 correção modular, L2 correção quadratica
from keras.regularizers import L1,L2


input_shape = X_train.iloc[0].shape
reset_seeds()
model = Sequential()
model.add(Dense(units=100, activation='relu', input_shape=input_shape))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=300, activation='relu', kernel_regularizer=L1(0.001))) #colocando uma penalização L1 bem pequena
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=300, activation='relu',kernel_regularizer=L1(0.001))) #colocando uma penalização L1 bem pequena
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=3, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              metrics=['accuracy'])

print(model.summary())
hist = model.fit(x=X_train,
                 y=y_train,
                 validation_split = 0.2, #Aqui posso acrescentar pro keras validar um percentual
                 epochs=30,
                 verbose = 3)

plot_convergence(
    hist=hist,
    metric ='loss'
    )

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Model: "sequential_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_72 (Dense)                │ (None, 100)            │         2,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_73 (Dense)                │ (None, 100)            │        10,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_74 (Dense)                │ (None, 100)            │        10,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_75 (Dense)                │ (None, 300)            │        30,300 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_76 (Dense)                │ (None, 100)            │        30,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_77 (Dense)                │ (None, 300)            │        30,300 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_78 (Dense)                │ (None, 100)            │        30,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_79 (Dense)                │ (None, 3)              │           303 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 143,503 (560.56 KB)

 Trainable params: 143,503 (560.56 KB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/30
Epoch 2/30
Epoch 3/30
Epoch 4/30
Epoch 5/30
Epoch 6/30
Epoch 7/30
Epoch 8/30
Epoch 9/30
Epoch 10/30
Epoch 11/30
Epoch 12/30
Epoch 13/30
Epoch 14/30
Epoch 15/30
Epoch 16/30
Epoch 17/30
Epoch 18/30
Epoch 19/30
Epoch 20/30
Epoch 21/30
Epoch 22/30
Epoch 23/30
Epoch 24/30
Epoch 25/30
Epoch 26/30
Epoch 27/30
Epoch 28/30
Epoch 29/30
Epoch 30/30


In [ ]:
'''
 é possível verificar que houve uma qtd de 143k de parametros, mas a penalização fez com que o modelo convergisse para o decrescimo da função de perda da validação
 ao aumentar a taxa de regularização (kernel_regularizer) é possível veriicar que fica mais dificil, tem que cuidar.
'''

'\n é possível verificar que houve uma qtd de 143k de parametros, mas a penalização fez com que o modelo convergisse para o decrescimo da função de perda da validação\n ao aumentar a taxa de regularização (kernel_regularizer) é possível veriicar que fica mais dificil, tem que cuidar.\n'

In [ ]:
#PENALIZAÇÂO DE DROPOUT (matando neuronios) -- Só posso fazer na parte de treinamento do modelo!!!!!!!!!!!!!!!!!!!!!

#adicionando uma camada de dropout depois da camada que queremos aplicar (poderia colocar entre as camadas que têm o maior conhecimento)


input_shape = X_train.iloc[0].shape
reset_seeds()
model = Sequential()
model.add(Dense(units=100, activation='relu', input_shape=input_shape))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=300, activation='relu'))
model.add(Dropout(0.4)) #aqui acrescentamos o dropout - taxas entre 20 a 50% tende a evitar esse super ajuste- mas n é certeza, tem que testar
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=300, activation='relu'))
model.add(Dropout(0.4))#aqui acrescentamos o dropout -
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=3, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              metrics=['accuracy'])

print(model.summary())
hist = model.fit(x=X_train,
                 y=y_train,
                 validation_split = 0.2, #Aqui posso acrescentar pro keras validar um percentual
                 epochs=30,
                 verbose = 3)

plot_convergence(
    hist=hist,
    metric ='loss'
    )

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Model: "sequential_15"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_88 (Dense)                │ (None, 100)            │         2,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_89 (Dense)                │ (None, 100)            │        10,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_90 (Dense)                │ (None, 100)            │        10,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_91 (Dense)                │ (None, 300)            │        30,300 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 300)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_92 (Dense)                │ (None, 100)            │        30,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_93 (Dense)                │ (None, 300)            │        30,300 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 300)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_94 (Dense)                │ (None, 100)            │        30,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_95 (Dense)                │ (None, 3)              │           303 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 143,503 (560.56 KB)

 Trainable params: 143,503 (560.56 KB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/30
Epoch 2/30
Epoch 3/30
Epoch 4/30
Epoch 5/30
Epoch 6/30
Epoch 7/30
Epoch 8/30
Epoch 9/30
Epoch 10/30
Epoch 11/30
Epoch 12/30
Epoch 13/30
Epoch 14/30
Epoch 15/30
Epoch 16/30
Epoch 17/30
Epoch 18/30
Epoch 19/30
Epoch 20/30
Epoch 21/30
Epoch 22/30
Epoch 23/30
Epoch 24/30
Epoch 25/30
Epoch 26/30
Epoch 27/30
Epoch 28/30
Epoch 29/30
Epoch 30/30


In [ ]:
'''aqui manteve-se a mesma qts de parametros e o modelo ainda está com overfitting
'''
#aumentar a taxa e aplicar em mais camadas

input_shape = X_train.iloc[0].shape
reset_seeds()
model = Sequential()
model.add(Dense(units=100, activation='relu', input_shape=input_shape))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=300, activation='relu'))
model.add(Dropout(0.6)) #aqui acrescentamos o dropout - taxas entre 20 a 50% tende a evitar esse super ajuste- mas n é certeza, tem que testar
model.add(Dense(units=100, activation='relu'))
model.add(Dropout(0.3)) #posso variar o percentual aqui tb
model.add(Dense(units=300, activation='relu'))
model.add(Dropout(0.6))#aqui acrescentamos o dropout -
model.add(Dense(units=100, activation='relu'))
model.add(Dropout(0.3))#posso variar o percentual aqui tb
model.add(Dense(units=3, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              metrics=['accuracy'])

print(model.summary())
hist = model.fit(x=X_train,
                 y=y_train,
                 validation_split = 0.2, #Aqui posso acrescentar pro keras validar um percentual
                 epochs=30,
                 verbose = 3)

plot_convergence(
    hist=hist,
    metric ='loss'
    )

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Model: "sequential_16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_96 (Dense)                │ (None, 100)            │         2,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_97 (Dense)                │ (None, 100)            │        10,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_98 (Dense)                │ (None, 100)            │        10,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_99 (Dense)                │ (None, 300)            │        30,300 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 300)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_100 (Dense)               │ (None, 100)            │        30,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_101 (Dense)               │ (None, 300)            │        30,300 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 300)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_102 (Dense)               │ (None, 100)            │        30,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_103 (Dense)               │ (None, 3)              │           303 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 143,503 (560.56 KB)

 Trainable params: 143,503 (560.56 KB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/30
Epoch 2/30
Epoch 3/30
Epoch 4/30
Epoch 5/30
Epoch 6/30
Epoch 7/30
Epoch 8/30
Epoch 9/30
Epoch 10/30
Epoch 11/30
Epoch 12/30
Epoch 13/30
Epoch 14/30
Epoch 15/30
Epoch 16/30
Epoch 17/30
Epoch 18/30
Epoch 19/30
Epoch 20/30
Epoch 21/30
Epoch 22/30
Epoch 23/30
Epoch 24/30
Epoch 25/30
Epoch 26/30
Epoch 27/30
Epoch 28/30
Epoch 29/30
Epoch 30/30


In [ ]:
# AINDA DEU RUIM!
##colocando um dropout para cada camada do modelo

input_shape = X_train.iloc[0].shape
reset_seeds()
model = Sequential()
model.add(Dense(units=100, activation='relu', input_shape=input_shape))
model.add(Dropout(0.3))#posso variar o percentual aqui tb
model.add(Dense(units=100, activation='relu'))
model.add(Dropout(0.3))#posso variar o percentual aqui tb
model.add(Dense(units=100, activation='relu'))
model.add(Dropout(0.3))#posso variar o percentual aqui tb
model.add(Dense(units=300, activation='relu'))
model.add(Dropout(0.6)) #aqui acrescentamos o dropout - taxas entre 20 a 50% tende a evitar esse super ajuste- mas n é certeza, tem que testar
model.add(Dense(units=100, activation='relu'))
model.add(Dropout(0.3)) #posso variar o percentual aqui tb
model.add(Dense(units=300, activation='relu'))
model.add(Dropout(0.6))#aqui acrescentamos o dropout -
model.add(Dense(units=100, activation='relu'))
model.add(Dropout(0.3))#posso variar o percentual aqui tb
model.add(Dense(units=3, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              metrics=['accuracy'])

print(model.summary())
hist = model.fit(x=X_train,
                 y=y_train,
                 validation_split = 0.2, #Aqui posso acrescentar pro keras validar um percentual
                 epochs=30,
                 verbose = 3)

plot_convergence(
    hist=hist,
    metric ='loss'
    )

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Model: "sequential_17"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_104 (Dense)               │ (None, 100)            │         2,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_105 (Dense)               │ (None, 100)            │        10,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_106 (Dense)               │ (None, 100)            │        10,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_107 (Dense)               │ (None, 300)            │        30,300 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 300)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_108 (Dense)               │ (None, 100)            │        30,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_109 (Dense)               │ (None, 300)            │        30,300 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_13 (Dropout)            │ (None, 300)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_110 (Dense)               │ (None, 100)            │        30,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_14 (Dropout)            │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_111 (Dense)               │ (None, 3)              │           303 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 143,503 (560.56 KB)

 Trainable params: 143,503 (560.56 KB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/30
Epoch 2/30
Epoch 3/30
Epoch 4/30
Epoch 5/30
Epoch 6/30
Epoch 7/30
Epoch 8/30
Epoch 9/30
Epoch 10/30
Epoch 11/30
Epoch 12/30
Epoch 13/30
Epoch 14/30
Epoch 15/30
Epoch 16/30
Epoch 17/30
Epoch 18/30
Epoch 19/30
Epoch 20/30
Epoch 21/30
Epoch 22/30
Epoch 23/30
Epoch 24/30
Epoch 25/30
Epoch 26/30
Epoch 27/30
Epoch 28/30
Epoch 29/30
Epoch 30/30


In [ ]:
#OOOPA, DEU BOAAAA

In [ ]:

###OBS: posso usar as duas penalizações em conjunto

input_shape = X_train.iloc[0].shape
reset_seeds()
model = Sequential()
model.add(Dense(units=100, activation='relu', input_shape=input_shape))
model.add(Dropout(0.3))#posso variar o percentual aqui tb
model.add(Dense(units=100, activation='relu'))
model.add(Dropout(0.3))#posso variar o percentual aqui tb
model.add(Dense(units=100, activation='relu'))
model.add(Dropout(0.3))#posso variar o percentual aqui tb
model.add(Dense(units=300, activation='relu', kernel_regularizerL1(0.0001))) # posso colocar aqui
model.add(Dropout(0.6)) #aqui acrescentamos o dropout - taxas entre 20 a 50% tende a evitar esse super ajuste- mas n é certeza, tem que testar
model.add(Dense(units=100, activation='relu'))
model.add(Dropout(0.3)) #posso variar o percentual aqui tb
model.add(Dense(units=300, activation='relu'))
model.add(Dropout(0.6))#aqui acrescentamos o dropout -
model.add(Dense(units=100, activation='relu'))
model.add(Dropout(0.3))#posso variar o percentual aqui tb
model.add(Dense(units=3, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              metrics=['accuracy'])

print(model.summary())
hist = model.fit(x=X_train,
                 y=y_train,
                 validation_split = 0.2, #Aqui posso acrescentar pro keras validar um percentual
                 epochs=30,
                 verbose = 3)

plot_convergence(
    hist=hist,
    metric ='loss'
    )


SyntaxError: positional argument follows keyword argument (ipython-input-56-3513594010.py, line 12)